# Fall Detection Study based on RNNs

The results of the research have been presented and described in the paper titled **"Fall detection from accelerometer data using recurrent neural networks"**

**Authors:** Natalia Bartczak, Marta Glanowska, Karolina Kowalewicz, Maciej Kunin, Robert Susik

In [1]:
#!pip install scikit-learn

In [2]:
import scipy.io
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import KFold

In [3]:
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Load data

In [5]:
fall_data = scipy.io.loadmat('./UniMiB-SHAR/data/fall_data.mat')['fall_data']
adl_data  = scipy.io.loadmat('./UniMiB-SHAR/data/adl_data.mat')['adl_data']

In [6]:
fall_data = pd.DataFrame(fall_data)
adl_data = pd.DataFrame(adl_data)

In [7]:
fall_data.shape, adl_data.shape

((4192, 453), (7579, 453))

Load labels

In [8]:
fall_labels = scipy.io.loadmat('./UniMiB-SHAR/data/fall_labels.mat')['fall_labels']
fall_labels = pd.DataFrame(fall_labels)     #activity id, person id, trial number

adl_labels = scipy.io.loadmat('./UniMiB-SHAR/data/adl_labels.mat')['adl_labels']
adl_labels = pd.DataFrame(adl_labels)       #activity id, person id, trial number

In [9]:
fall_labels.shape, adl_labels.shape

((4192, 3), (7579, 3))

Prepare labels

In [10]:
fall_lab = fall_labels[1]
d = {'subject': fall_labels[1], 'label': list(np.ones(len(fall_labels[1])))}
df_fall = pd.DataFrame(data=d)

adl_lab = adl_labels[1]
d = {'subject': adl_labels[1], 'label': list(np.zeros(len(adl_labels[1])))}
df_adl = pd.DataFrame(data=d)

In [11]:
df_fall.shape, df_adl.shape

((4192, 2), (7579, 2))

## 1. Split to folds - stratified fold with respect to person number, proportional split (k = 5)

In [12]:
import random

In [13]:
fold_1_lab = pd.DataFrame()
fold_2_lab = pd.DataFrame()
fold_3_lab = pd.DataFrame()
fold_4_lab = pd.DataFrame()
fold_5_lab = pd.DataFrame()

fold_1 = pd.DataFrame()
fold_2 = pd.DataFrame()
fold_3 = pd.DataFrame()
fold_4 = pd.DataFrame()
fold_5 = pd.DataFrame()

In [14]:
random.seed(123)

In [15]:
#falls
for num_person in range(1, 31):
    num_per_person = df_fall[df_fall['subject'] == num_person].count()[0]
    idxs = df_fall[df_fall['subject'] == num_person].index.to_list()
    random.shuffle(idxs)                                                #added - random split
    k = df_fall[df_fall['subject'] == num_person].count()[0]//5
    fold_1_lab = pd.concat([fold_1_lab, df_fall.loc[idxs[:k]]['label']])
    fold_1 = pd.concat([fold_1, fall_data.loc[idxs[:k]]])
    fold_2_lab = pd.concat([fold_2_lab, df_fall.loc[idxs[k:k*2]]['label']])
    fold_2 = pd.concat([fold_2, fall_data.loc[idxs[k:k*2]]])
    fold_3_lab = pd.concat([fold_3_lab, df_fall.loc[idxs[k*2:k*3]]['label']])
    fold_3 = pd.concat([fold_3, fall_data.loc[idxs[k*2:k*3]]])
    fold_4_lab = pd.concat([fold_4_lab, df_fall.loc[idxs[k*3:k*4]]['label']])
    fold_4 = pd.concat([fold_4, fall_data.loc[idxs[k*3:k*4]]])
    fold_5_lab = pd.concat([fold_5_lab, df_fall.loc[idxs[k*4:k*5]]['label']])
    fold_5 = pd.concat([fold_5, fall_data.loc[idxs[k*4:k*5]]])

In [16]:
#adl
for num_person in range(1, 31):
    num_per_person = df_adl[df_adl['subject'] == num_person].count()[0]
    idxs = df_adl[df_adl['subject'] == num_person].index.to_list()
    random.shuffle(idxs)                                                #added - random split
    k = df_adl[df_adl['subject'] == num_person].count()[0]//5
    fold_1_lab = pd.concat([fold_1_lab, df_adl.loc[idxs[:k]]['label']])
    fold_1 = pd.concat([fold_1, adl_data.loc[idxs[:k]]])
    fold_2_lab = pd.concat([fold_2_lab, df_adl.loc[idxs[k:k*2]]['label']])
    fold_2 = pd.concat([fold_2, adl_data.loc[idxs[k:k*2]]])
    fold_3_lab = pd.concat([fold_3_lab, df_adl.loc[idxs[k*2:k*3]]['label']])
    fold_3 = pd.concat([fold_3, adl_data.loc[idxs[k*2:k*3]]])
    fold_4_lab = pd.concat([fold_4_lab, df_adl.loc[idxs[k*3:k*4]]['label']])
    fold_4 = pd.concat([fold_4, adl_data.loc[idxs[k*3:k*4]]])
    fold_5_lab = pd.concat([fold_5_lab, df_adl.loc[idxs[k*4:k*5]]['label']])
    fold_5 = pd.concat([fold_5, adl_data.loc[idxs[k*4:k*5]]])

Check

In [17]:
fold_1_lab

,0
49,1.0
91,1.0
19,1.0
92,1.0
119,1.0
...,...
7043,0.0
5753,0.0
2055,0.0
4073,0.0


In [18]:
#check - original dataset samples number
df_fall[df_fall['subject'] == 1].count()[0] + df_adl[df_adl['subject'] == 1].count()[0]

384

In [19]:
#all subjects, fold 1
fold_1_lab.value_counts()

0.0    1503
1.0     823
Name: count, dtype: int64

### CNN-infused model

In [18]:
def build_model():
    hidden_dim = 512
    seq_len = fold_1.shape[1]
    n_features = 1
    learning_rate = 0.0003

    regularizers = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
    gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
    rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
    cov1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1))
    max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
    cov2 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1))
    max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
    cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
    max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
    rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
    tdd1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2))
    f1   = tf.keras.layers.Flatten()(tdd1)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(f1)

    classifier = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier

Building own cross_validation model

https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md

In [22]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    #print(test_x.shape[0])
    #print(fold_1.shape[0]*4)
    classifier = build_model()

    
    history = classifier.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=(train_x.shape[0] // batch_size),
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00003)]
    )

    val_acc_per_fold.append(history.history['val_accuracy'])
    acc_per_fold.append(history.history['accuracy'])
    val_loss_per_fold.append(history.history['val_loss'])
    loss_per_fold.append(history.history['loss'])


Epoch 1/20
581/581 [==============================] - 162s 263ms/step - loss: 0.8191 - accuracy: 0.8523 - val_loss: 0.7482 - val_accuracy: 0.8160 - lr: 3.0000e-04
Epoch 2/20
581/581 [==============================] - 154s 263ms/step - loss: 0.4269 - accuracy: 0.9072 - val_loss: 0.4770 - val_accuracy: 0.8779 - lr: 3.0000e-04
Epoch 3/20
581/581 [==============================] - 152s 262ms/step - loss: 0.2540 - accuracy: 0.9480 - val_loss: 0.2779 - val_accuracy: 0.9295 - lr: 3.0000e-04
Epoch 4/20
581/581 [==============================] - 153s 263ms/step - loss: 0.1826 - accuracy: 0.9633 - val_loss: 0.3109 - val_accuracy: 0.9162 - lr: 3.0000e-04
Epoch 5/20
581/581 [==============================] - 153s 263ms/step - loss: 0.1333 - accuracy: 0.9759 - val_loss: 0.2376 - val_accuracy: 0.9291 - lr: 3.0000e-04
Epoch 6/20
581/581 [==============================] - 153s 263ms/step - loss: 0.1160 - accuracy: 0.9806 - val_loss: 0.2985 - val_accuracy: 0.9157 - lr: 3.0000e-04
Epoch 7/20
581/581 [==

In [23]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8159931302070618,
   0.8779019713401794,
   0.929492712020874,
   0.9161651134490967,
   0.9290627837181091,
   0.9157351851463318,
   0.9436801671981812,
   0.9217540621757507,
   0.9380911588668823,
   0.9819432497024536,
   0.9565778374671936,
   0.9772140979766846,
   0.960017204284668,
   0.9587274193763733,
   0.964746356010437,
   0.9505589008331299,
   0.9333620071411133,
   0.980223536491394,
   0.979363739490509,
   0.9711951613426208],
  [0.8310404419898987,
   0.9299226403236389,
   0.9617368578910828,
   0.9634565711021423,
   0.995270848274231,
   0.9866724014282227,
   0.9883921146392822,
   0.9914015531539917,
   0.9948409199714661,
   0.9896818399429321,
   0.9961307048797607,
   0.9978503584861755,
   0.9767841696739197,
   0.9987102150917053,
   0.9686156511306763,
   0.9987102150917053,
   0.9987102150917053,
   0.9987102150917053,
   0.9909716248512268,
   0.8336199522018433],
  [0.8925193548202515,
   0.9165950417518616,
   0.9673258662223816,
   0.9828031063

In [6]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model2.pickle', 'wb') as file:
 pickle.dump(results, file)

NameError: name 'val_acc_per_fold' is not defined

In [5]:
import pickle
with open('results_model2.pickle', 'rb') as file:
    data = pickle.load(file)

EOFError: Ran out of input

In [7]:
(0.9711951613426208 + 0.8336199522018433 + 0.9978503584861755 + 0.9337919354438782 + 0.9621667861938477)/5

0.9397248387336731

In [ ]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier.save("classifier")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier")

### Bidirectional RNN model

In [146]:
def build_model_bidirectional():
    hidden_dim = 512
    seq_len = fold_1.shape[1]
    n_features = 1
    learning_rate = 0.0003

    regularizers = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    regularizers_GRU = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5),
        'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU))(in1)
    bn = tf.keras.layers.BatchNormalization()(gru1)
    dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
    drop = tf.keras.layers.Dropout(0.3)(dense)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

    classifier_test_opt_bi = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    print(classifier_test_opt_bi.summary())

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier_test_opt_bi.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier_test_opt_bi

In [21]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    classifier_test_opt_bi = build_model_bidirectional()

    history_test_opt_bi = classifier_test_opt_bi.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=train_x.shape[0] // batch_size,
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
    )

    val_acc_per_fold.append(history_test_opt_bi.history['val_accuracy'])
    acc_per_fold.append(history_test_opt_bi.history['accuracy'])
    val_loss_per_fold.append(history_test_opt_bi.history['val_loss'])
    loss_per_fold.append(history_test_opt_bi.history['loss'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 453, 1)]          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 1024)             1582080   
 nal)                                                            
                                                                 
 batch_normalization_1 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 128)               131200    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129 

In [22]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8508168458938599,
   0.8766121864318848,
   0.9325021505355835,
   0.9359415173530579,
   0.9041272401809692,
   0.9277729988098145,
   0.9716250896453857,
   0.9720550179481506,
   0.9823731780052185,
   0.9711951613426208,
   0.9780739545822144,
   0.9724849462509155,
   0.970765233039856,
   0.9797936081886292,
   0.9780739545822144,
   0.9763542413711548,
   0.9763542413711548,
   0.9750645160675049,
   0.9767841696739197,
   0.9763542413711548],
  [0.8993980884552002,
   0.9329320788383484,
   0.9617368578910828,
   0.929492712020874,
   0.9883921146392822,
   0.9815133213996887,
   0.9849526882171631,
   0.9978503584861755,
   0.9995700716972351,
   0.9961307048797607,
   0.9982802867889404,
   0.9978503584861755,
   0.9991401433944702,
   0.9991401433944702,
   0.9982802867889404,
   0.9974204897880554,
   0.9987102150917053,
   0.9987102150917053,
   0.9982802867889404,
   0.9982802867889404],
  [0.8155632019042969,
   0.910146176815033,
   0.9836629629135132,
   0.9922614

In [23]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_bi.pickle', 'wb') as file:
 pickle.dump(results, file)

In [29]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/5

0.9763542413711548
0.9982802867889404
0.9965606331825256
0.9742046594619751
0.9492691159248352


0.9789337873458862

**Bidirectional RNN model - zmniejszenie learning rate**

In [19]:
def build_model_bidirectional():
    hidden_dim = 512
    seq_len = fold_1.shape[1]
    n_features = 1
    learning_rate = 0.00003

    regularizers = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    regularizers_GRU = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5),
        'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU))(in1)
    bn = tf.keras.layers.BatchNormalization()(gru1)
    dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
    drop = tf.keras.layers.Dropout(0.3)(dense)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

    classifier_test_opt_bi = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    #print(classifier_test_opt_bi.summary())

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier_test_opt_bi.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier_test_opt_bi

In [21]:
epochs = 40
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    classifier_test_opt_bi = build_model_bidirectional()

    history_test_opt_bi = classifier_test_opt_bi.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=train_x.shape[0] // batch_size,
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
    )

    val_acc_per_fold.append(history_test_opt_bi.history['val_accuracy'])
    acc_per_fold.append(history_test_opt_bi.history['accuracy'])
    val_loss_per_fold.append(history_test_opt_bi.history['val_loss'])
    loss_per_fold.append(history_test_opt_bi.history['loss'])

Epoch 1/40
581/581 [==============================] - 50s 83ms/step - loss: 0.5407 - accuracy: 0.7984 - val_loss: 0.4716 - val_accuracy: 0.8362 - lr: 3.0000e-05
Epoch 2/40
581/581 [==============================] - 48s 83ms/step - loss: 0.4667 - accuracy: 0.8411 - val_loss: 0.4446 - val_accuracy: 0.8543 - lr: 3.0000e-05
Epoch 3/40
581/581 [==============================] - 48s 83ms/step - loss: 0.4474 - accuracy: 0.8480 - val_loss: 0.4641 - val_accuracy: 0.8336 - lr: 3.0000e-05
Epoch 4/40
581/581 [==============================] - 48s 82ms/step - loss: 0.4408 - accuracy: 0.8526 - val_loss: 0.4237 - val_accuracy: 0.8633 - lr: 3.0000e-05
Epoch 5/40
581/581 [==============================] - 48s 82ms/step - loss: 0.4246 - accuracy: 0.8622 - val_loss: 0.4073 - val_accuracy: 0.8646 - lr: 3.0000e-05
Epoch 6/40
581/581 [==============================] - 48s 83ms/step - loss: 0.4111 - accuracy: 0.8670 - val_loss: 0.3900 - val_accuracy: 0.8719 - lr: 3.0000e-05
Epoch 7/40
581/581 [==============

In [22]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8361994624137878,
   0.8542562127113342,
   0.8336199522018433,
   0.8632845878601074,
   0.8645743727684021,
   0.8718830347061157,
   0.8852106332778931,
   0.8890799880027771,
   0.8989681601524353,
   0.8993980884552002,
   0.9019776582717896,
   0.9174548387527466,
   0.9363714456558228,
   0.945399820804596,
   0.9544281959533691,
   0.9544281959533691,
   0.9630266427993774,
   0.9582974910736084,
   0.9664660096168518,
   0.970765233039856,
   0.9724849462509155,
   0.9789338111877441,
   0.9772140979766846,
   0.9750645160675049,
   0.9785038828849792,
   0.9858125448226929,
   0.9858125448226929,
   0.9832330346107483,
   0.9845227599143982,
   0.9871023297309875,
   0.9875322580337524,
   0.9875322580337524,
   0.9875322580337524,
   0.9875322580337524,
   0.9871023297309875,
   0.9879621863365173,
   0.9883921146392822,
   0.9879621863365173,
   0.9879621863365173,
   0.9875322580337524],
  [0.8576956391334534,
   0.8701633810997009,
   0.8667240142822266,
   0.8667240

In [23]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_bi_small_lr.pickle', 'wb') as file:
 pickle.dump(results, file)

In [24]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/5

0.9875322580337524
0.985382616519928
0.9819432497024536
0.9883921146392822
0.9806534647941589


0.984780740737915

In [25]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier_test_opt_bi.save("classifier_test_opt_bi_small_lr")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier_test_opt_bi_small_lr")

INFO:tensorflow:Assets written to: classifier_test_opt_bi_small_lr\assets


INFO:tensorflow:Assets written to: classifier_test_opt_bi_small_lr\assets


In [29]:
reconstructed_model = tf.keras.models.load_model("classifier_test_opt_bi_small_lr")

In [31]:
reconstructed_model.predict(np.array(fold_5))

73/73 [==============================] - 3s 34ms/step


array([[0.9996544 ],
       [0.9971124 ],
       [0.98283744],
       ...,
       [0.00115953],
       [0.00104059],
       [0.05627833]], dtype=float32)

**Bidirectional RNN model - randomly created folds, regular lr**

In [170]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    classifier_test_opt_bi = build_model_bidirectional()

    history_test_opt_bi = classifier_test_opt_bi.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=train_x.shape[0] // batch_size,
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-8)]
    )

    val_acc_per_fold.append(history_test_opt_bi.history['val_accuracy'])
    acc_per_fold.append(history_test_opt_bi.history['accuracy'])
    val_loss_per_fold.append(history_test_opt_bi.history['val_loss'])
    loss_per_fold.append(history_test_opt_bi.history['loss'])

Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 453, 1)]          0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 1024)             1582080   
 nal)                                                            
                                                                 
 batch_normalization_15 (Bat  (None, 1024)             4096      
 chNormalization)                                                
                                                                 
 dense_30 (Dense)            (None, 128)               131200    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_31 (Dense)            (None, 1)                 129

In [171]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.850386917591095,
   0.9256234169006348,
   0.8890799880027771,
   0.9282029271125793,
   0.97463458776474,
   0.9630266427993774,
   0.9806534647941589,
   0.9871023297309875,
   0.985382616519928,
   0.9905416965484619,
   0.9935511350631714,
   0.9918314814567566,
   0.9939810633659363,
   0.9819432497024536,
   0.9931212663650513,
   0.9957007765769958,
   0.9978503584861755,
   0.9957007765769958,
   0.9961307048797607,
   0.9978503584861755],
  [0.8817712664604187,
   0.9393808841705322,
   0.9079965353012085,
   0.9135855436325073,
   0.9742046594619751,
   0.9823731780052185,
   0.9785038828849792,
   0.9862424731254578,
   0.9918314814567566,
   0.9888219833374023,
   0.9492691159248352,
   0.9961307048797607,
   0.9914015531539917,
   0.9961307048797607,
   0.9965606331825256,
   0.9948409199714661,
   0.9961307048797607,
   0.9957007765769958,
   0.9961307048797607,
   0.9969905614852905],
  [0.869733452796936,
   0.9273430705070496,
   0.9411005973815918,
   0.966466009

In [176]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_bi_random_folds.pickle', 'wb') as file:
 pickle.dump(results, file)

In [173]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/5

0.9978503584861755
0.9969905614852905
0.9991401433944702
0.9931212663650513
0.9978503584861755


0.9969905376434326

In [175]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier_test_opt_bi.save("classifier_test_opt_bi_random_folds")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier_RNN_random_folds")

INFO:tensorflow:Assets written to: classifier_test_opt_bi_random_folds\assets


INFO:tensorflow:Assets written to: classifier_test_opt_bi_random_folds\assets


### Simplified RNN model

In [120]:
def build_model_RNN():
    hidden_dim = 512
    seq_len = fold_1.shape[1]
    n_features = 1
    learning_rate = 0.0003

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer')(in1)
    bn = tf.keras.layers.BatchNormalization()(gru1)
    dense = tf.keras.layers.Dense(128, activation="relu")(bn)
    drop = tf.keras.layers.Dropout(0.3)(dense)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid')(drop)

    classifier_RNN = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    #print(classifier_RNN.summary())

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier_RNN.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier_RNN

In [65]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    classifier_RNN = build_model_RNN()

    history_RNN = classifier_RNN.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=train_x.shape[0] // batch_size,
        validation_data=(np.array(test_x), np.array(test_y))
    )

    val_acc_per_fold.append(history_RNN.history['val_accuracy'])
    acc_per_fold.append(history_RNN.history['accuracy'])
    val_loss_per_fold.append(history_RNN.history['val_loss'])
    loss_per_fold.append(history_RNN.history['loss'])

Epoch 1/20
581/581 [==============================] - 33s 53ms/step - loss: 0.4237 - accuracy: 0.8343 - val_loss: 0.5230 - val_accuracy: 0.8401
Epoch 2/20
581/581 [==============================] - 30s 51ms/step - loss: 0.3276 - accuracy: 0.8767 - val_loss: 0.4277 - val_accuracy: 0.8826
Epoch 3/20
581/581 [==============================] - 30s 52ms/step - loss: 0.2105 - accuracy: 0.9184 - val_loss: 0.5029 - val_accuracy: 0.7971
Epoch 4/20
581/581 [==============================] - 30s 52ms/step - loss: 0.1621 - accuracy: 0.9397 - val_loss: 0.2906 - val_accuracy: 0.8938
Epoch 5/20
581/581 [==============================] - 31s 53ms/step - loss: 0.1620 - accuracy: 0.9360 - val_loss: 0.0864 - val_accuracy: 0.9647
Epoch 6/20
581/581 [==============================] - 30s 52ms/step - loss: 0.0810 - accuracy: 0.9727 - val_loss: 0.1456 - val_accuracy: 0.9342
Epoch 7/20
581/581 [==============================] - 30s 51ms/step - loss: 0.1324 - accuracy: 0.9540 - val_loss: 0.1261 - val_accuracy:

In [66]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8400688171386719,
   0.8826311230659485,
   0.7970765233039856,
   0.8938091397285461,
   0.964746356010437,
   0.9342218637466431,
   0.9522786140441895,
   0.97463458776474,
   0.9845227599143982,
   0.9858125448226929,
   0.9914015531539917,
   0.9892519116401672,
   0.9815133213996887,
   0.9879621863365173,
   0.9888219833374023,
   0.9806534647941589,
   0.990111768245697,
   0.9892519116401672,
   0.9750645160675049,
   0.9849526882171631],
  [0.904987096786499,
   0.8822011947631836,
   0.8946689367294312,
   0.9703353643417358,
   0.7665520310401917,
   0.8078246116638184,
   0.845227837562561,
   0.8478074073791504,
   0.9187446236610413,
   0.935511589050293,
   0.9428203105926514,
   0.9441100358963013,
   0.9591573476791382,
   0.969905436038971,
   0.954858124256134,
   0.985382616519928,
   0.9737747311592102,
   0.9888219833374023,
   0.9840928912162781,
   0.9957007765769958],
  [0.90025794506073,
   0.9144454002380371,
   0.9578675627708435,
   0.985382616519928,

In [67]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_rnn.pickle', 'wb') as file:
 pickle.dump(results, file)

In [68]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/5

0.9849526882171631
0.9957007765769958
0.8512467741966248
0.9836629629135132
0.9656062126159668


0.9562338829040528

**Simplified RNN model - randomly created folds, regular lr**

In [121]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in range(len(sets)):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    classifier_RNN = build_model_RNN()

    history_RNN = classifier_RNN.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=train_x.shape[0] // batch_size,
        validation_data=(np.array(test_x), np.array(test_y))
    )

    val_acc_per_fold.append(history_RNN.history['val_accuracy'])
    acc_per_fold.append(history_RNN.history['accuracy'])
    val_loss_per_fold.append(history_RNN.history['val_loss'])
    loss_per_fold.append(history_RNN.history['loss'])

Epoch 1/20
581/581 [==============================] - 26s 42ms/step - loss: 0.4345 - accuracy: 0.8234 - val_loss: 0.3714 - val_accuracy: 0.8654
Epoch 2/20
581/581 [==============================] - 24s 42ms/step - loss: 0.3121 - accuracy: 0.8814 - val_loss: 0.3268 - val_accuracy: 0.8629
Epoch 3/20
581/581 [==============================] - 25s 42ms/step - loss: 0.1752 - accuracy: 0.9385 - val_loss: 0.1379 - val_accuracy: 0.9467
Epoch 4/20
581/581 [==============================] - 24s 42ms/step - loss: 0.1173 - accuracy: 0.9591 - val_loss: 0.0788 - val_accuracy: 0.9699
Epoch 5/20
581/581 [==============================] - 24s 42ms/step - loss: 0.0784 - accuracy: 0.9714 - val_loss: 0.0370 - val_accuracy: 0.9880
Epoch 6/20
581/581 [==============================] - 24s 42ms/step - loss: 0.0594 - accuracy: 0.9811 - val_loss: 0.0347 - val_accuracy: 0.9888
Epoch 7/20
581/581 [==============================] - 24s 42ms/step - loss: 0.0511 - accuracy: 0.9836 - val_loss: 0.0505 - val_accuracy:

In [122]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8654342293739319,
   0.8628546595573425,
   0.9466896057128906,
   0.969905436038971,
   0.9879621863365173,
   0.9888219833374023,
   0.9832330346107483,
   0.990111768245697,
   0.9939810633659363,
   0.9914015531539917,
   0.9815133213996887,
   0.9935511350631714,
   0.9905416965484619,
   0.9939810633659363,
   0.9819432497024536,
   0.9948409199714661,
   0.9909716248512268,
   0.9957007765769958,
   0.990111768245697,
   0.9961307048797607],
  [0.8817712664604187,
   0.794497013092041,
   0.9496990442276001,
   0.970765233039856,
   0.9767841696739197,
   0.9759243130683899,
   0.9767841696739197,
   0.9840928912162781,
   0.9918314814567566,
   0.995270848274231,
   0.9862424731254578,
   0.9935511350631714,
   0.9965606331825256,
   0.9922614097595215,
   0.9982802867889404,
   0.9879621863365173,
   0.9961307048797607,
   0.9939810633659363,
   0.9969905614852905,
   0.9978503584861755],
  [0.8718830347061157,
   0.9071367383003235,
   0.889509916305542,
   0.92304384708

In [123]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_rnn_random_folds.pickle', 'wb') as file:
 pickle.dump(results, file)

In [124]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/5

0.9961307048797607
0.9978503584861755
0.9991401433944702
0.995270848274231
0.9961307048797607


0.9969045519828796

In [125]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier_RNN.save("classifier_RNN_random_folds")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier_RNN_random_folds")

INFO:tensorflow:Assets written to: classifier_RNN_random_folds\assets


INFO:tensorflow:Assets written to: classifier_RNN_random_folds\assets


## 2. Second approach - Leave One Out

### CNN-infused model

In [13]:
# def build_model():
#     hidden_dim = 512
#     seq_len = train_x.shape[1]
#     n_features = 1
#     learning_rate = 0.0003

#     regularizers = {
#         'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
#         'bias_regularizer':tf.keras.regularizers.l2(1e-4),
#         'activity_regularizer':tf.keras.regularizers.l2(1e-5)
#     }

#     in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
#     gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, return_sequences=True, **regularizers))(in1)
#     gru2 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers))(gru1)
#     rsh1 = tf.keras.layers.Reshape( (hidden_dim*2, 1) )(gru2)
#     cov1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(rsh1))
#     max1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov1)
#     cov2 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 5, activation='relu', **regularizers)(max1))
#     max2 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov2)
#     cov3 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.Conv1D(seq_len, 3, activation='relu', **regularizers)(max2))
#     max3 = tf.keras.layers.MaxPool1D(pool_size=3, strides=1)(cov3)
#     rsh2 = tf.keras.layers.Reshape( (seq_len, 1010) )(max3)
#     tdd1 = tf.keras.layers.Dropout(0.1)(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,**regularizers))(rsh2))
#     f1   = tf.keras.layers.Flatten()(tdd1)
#     d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(f1)

#     classifier = tf.keras.Model(
#         inputs=[in1], 
#         outputs=[d1]
#     )

#     opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#     classifier.compile(loss='binary_crossentropy', optimizer=opt, 
#         metrics=['accuracy'])
    
#     return classifier

In [ ]:
# epochs = 20
# batch_size = 16

# val_acc_per_fold = []
# acc_per_fold = []
# val_loss_per_fold = []
# loss_per_fold = []

# for n in range(1, 31):
#     test_x = pd.DataFrame()
#     test_y = pd.DataFrame()

#     idxs_test = df_fall[df_fall['subject'] == n].index.to_list()
#     idxs_test_2 = df_adl[df_adl['subject'] == n].index.to_list()
#     test_x = pd.concat([test_x, fall_data.loc[idxs_test], adl_data.loc[idxs_test_2]])
#     test_y = pd.concat([test_y, df_fall.loc[idxs_test]['label'], df_adl.loc[idxs_test_2]['label']])

#     train_x = pd.DataFrame()
#     train_y = pd.DataFrame()

#     fall_data_c = fall_data.copy()
#     adl_data_c = adl_data.copy()
#     fall_data_c.drop(idxs_test, axis=0, inplace=True)
#     adl_data_c.drop(idxs_test_2, axis=0, inplace=True)
#     train_x = pd.concat([train_x, fall_data_c, adl_data_c])

#     df_fall_c = df_fall.copy()
#     df_adl_c = df_adl.copy()
#     df_fall_c.drop(idxs_test, axis=0, inplace=True)
#     df_adl_c.drop(idxs_test_2, axis=0, inplace=True)
#     train_y = pd.concat([train_y, df_fall_c['label'], df_adl_c['label']])

#     classifier = build_model()
    
#     history = classifier.fit(
#         np.array(train_x), np.array(train_y),
#         epochs=epochs,
#         batch_size = batch_size,
#         steps_per_epoch=(train_x[0]) // batch_size,
#         validation_data=(np.array(test_x), np.array(test_y)),
#         callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00003)]
#     )

#     val_acc_per_fold.append(history.history['val_accuracy'])
#     acc_per_fold.append(history.history['accuracy'])
#     val_loss_per_fold.append(history.history['val_loss'])
#     loss_per_fold.append(history.history['loss'])

### Bidirectional RNN model

In [20]:
def build_model_bidirectional():
    hidden_dim = 512
    seq_len = train_x.shape[1]
    n_features = 1
    learning_rate = 0.0003

    regularizers = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    regularizers_GRU = {
        'kernel_regularizer':tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4),
        'bias_regularizer':tf.keras.regularizers.l2(1e-4),
        'activity_regularizer':tf.keras.regularizers.l2(1e-5),
        'recurrent_regularizer':tf.keras.regularizers.l2(1e-5)
    }

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(hidden_dim, name='latent_layer', **regularizers_GRU))(in1)
    bn = tf.keras.layers.BatchNormalization()(gru1)
    dense = tf.keras.layers.Dense(128, activation="relu", **regularizers)(bn)
    drop = tf.keras.layers.Dropout(0.3)(dense)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid', **regularizers)(drop)

    classifier_test_opt_bi = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    #print(classifier_test_opt_bi.summary())

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier_test_opt_bi.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier_test_opt_bi

In [21]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []

for n in range(1, 31):
    test_x = pd.DataFrame()
    test_y = pd.DataFrame()

    idxs_test = df_fall[df_fall['subject'] == n].index.to_list()
    idxs_test_2 = df_adl[df_adl['subject'] == n].index.to_list()
    test_x = pd.concat([test_x, fall_data.loc[idxs_test], adl_data.loc[idxs_test_2]])
    test_y = pd.concat([test_y, df_fall.loc[idxs_test]['label'], df_adl.loc[idxs_test_2]['label']])
    print(test_x.shape, test_y.shape)

    train_x = pd.DataFrame()
    train_y = pd.DataFrame()

    fall_data_c = fall_data.copy()
    adl_data_c = adl_data.copy()
    fall_data_c.drop(idxs_test, axis=0, inplace=True)
    adl_data_c.drop(idxs_test_2, axis=0, inplace=True)
    train_x = pd.concat([train_x, fall_data_c, adl_data_c])

    df_fall_c = df_fall.copy()
    df_adl_c = df_adl.copy()
    df_fall_c.drop(idxs_test, axis=0, inplace=True)
    df_adl_c.drop(idxs_test_2, axis=0, inplace=True)
    train_y = pd.concat([train_y, df_fall_c['label'], df_adl_c['label']])
    #print(train_x.shape, train_y.shape)

    classifier_test_opt_bi_leave = build_model_bidirectional()

    #print(np.array(train_y.astype(int)))
    
    history_test_opt_bi_leave = classifier_test_opt_bi_leave.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=(train_x.shape[0]) // batch_size,
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00003)]
    )

    val_acc_per_fold.append(history_test_opt_bi_leave.history['val_accuracy'])
    acc_per_fold.append(history_test_opt_bi_leave.history['accuracy'])
    val_loss_per_fold.append(history_test_opt_bi_leave.history['val_loss'])
    loss_per_fold.append(history_test_opt_bi_leave.history['loss'])

(384, 453) (384, 1)
Epoch 1/20
711/711 [==============================] - 59s 76ms/step - loss: 0.5197 - accuracy: 0.8223 - val_loss: 0.4512 - val_accuracy: 0.8438 - lr: 3.0000e-04
Epoch 2/20
711/711 [==============================] - 54s 76ms/step - loss: 0.3699 - accuracy: 0.8858 - val_loss: 0.2559 - val_accuracy: 0.9349 - lr: 3.0000e-04
Epoch 3/20
711/711 [==============================] - 54s 76ms/step - loss: 0.2569 - accuracy: 0.9363 - val_loss: 0.2123 - val_accuracy: 0.9635 - lr: 3.0000e-04
Epoch 4/20
711/711 [==============================] - 54s 76ms/step - loss: 0.2384 - accuracy: 0.9439 - val_loss: 0.1278 - val_accuracy: 0.9896 - lr: 3.0000e-04
Epoch 5/20
711/711 [==============================] - 54s 76ms/step - loss: 0.1414 - accuracy: 0.9811 - val_loss: 0.1288 - val_accuracy: 0.9844 - lr: 3.0000e-04
Epoch 6/20
711/711 [==============================] - 54s 76ms/step - loss: 0.1263 - accuracy: 0.9850 - val_loss: 0.4077 - val_accuracy: 0.9089 - lr: 3.0000e-04
Epoch 7/20
711

In [22]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.84375,
   0.9348958134651184,
   0.9635416865348816,
   0.9895833134651184,
   0.984375,
   0.9088541865348816,
   0.9973958134651184,
   0.953125,
   0.9817708134651184,
   0.9791666865348816,
   0.9505208134651184,
   0.9973958134651184,
   0.9947916865348816,
   1.0,
   0.9947916865348816,
   0.9947916865348816,
   1.0,
   0.9583333134651184,
   1.0,
   1.0],
  [0.8233276009559631,
   0.8747856020927429,
   0.9914236664772034,
   0.9862778782844543,
   0.9554030895233154,
   0.8010291457176208,
   0.9931389093399048,
   1.0,
   0.9073756337165833,
   0.9982847571372986,
   0.994854211807251,
   0.994854211807251,
   0.9879931211471558,
   1.0,
   0.994854211807251,
   0.9879931211471558,
   0.9794167876243591,
   0.994854211807251,
   0.994854211807251,
   0.9982847571372986],
  [0.9177631735801697,
   0.9144737124443054,
   0.9671052694320679,
   0.9703947305679321,
   0.9144737124443054,
   0.9802631735801697,
   0.9934210777282715,
   0.9934210777282715,
   0.973684191703796

In [23]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_bi_leave_one_out.pickle', 'wb') as file:
 pickle.dump(results, file)

In [24]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/30

1.0
0.9982847571372986
1.0
0.9228295683860779
1.0
1.0
1.0
0.9294710159301758
0.987922728061676
1.0
0.9872449040412903
1.0
0.9628646969795227
0.9973118305206299
0.9974683523178101
0.9871794581413269
1.0
0.9896551966667175
1.0
1.0
0.9964285492897034
0.9896694421768188
0.9969325065612793
0.9819967150688171
0.990123450756073
0.9851190447807312
0.9971910119056702
1.0
0.9980582594871521
1.0


0.9898583829402924

#poprzednio: 0.98176

In [25]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier_test_opt_bi_leave.save("classifier_test_opt_bi_leave_one_out")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier_RNN_leave_one_out")

INFO:tensorflow:Assets written to: classifier_test_opt_bi_leave_one_out\assets


INFO:tensorflow:Assets written to: classifier_test_opt_bi_leave_one_out\assets


### Simplified RNN model

In [56]:
def build_model_RNN():
    hidden_dim = 512
    seq_len = fold_1.shape[1]
    n_features = 1
    learning_rate = 0.0003

    in1 = tf.keras.layers.Input(shape=(seq_len, n_features))
    gru1 = tf.keras.layers.GRU(hidden_dim, name='latent_layer')(in1)
    bn = tf.keras.layers.BatchNormalization()(gru1)
    dense = tf.keras.layers.Dense(128, activation="relu")(bn)
    drop = tf.keras.layers.Dropout(0.3)(dense)
    d1   = tf.keras.layers.Dense(1, activation='sigmoid')(drop)

    classifier_RNN = tf.keras.Model(
        inputs=[in1], 
        outputs=[d1]
    )

    #print(classifier_RNN.summary())

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    classifier_RNN.compile(loss='binary_crossentropy', optimizer=opt, 
        metrics=['accuracy'])
    
    return classifier_RNN

In [57]:
epochs = 20
batch_size = 16

val_acc_per_fold = []
acc_per_fold = []
val_loss_per_fold = []
loss_per_fold = []

for n in range(1, 31):
    test_x = pd.DataFrame()
    test_y = pd.DataFrame()

    idxs_test = df_fall[df_fall['subject'] == n].index.to_list()
    idxs_test_2 = df_adl[df_adl['subject'] == n].index.to_list()
    test_x = pd.concat([test_x, fall_data.loc[idxs_test], adl_data.loc[idxs_test_2]])
    test_y = pd.concat([test_y, df_fall.loc[idxs_test]['label'], df_adl.loc[idxs_test_2]['label']])
    #print(test_x.shape, test_y.shape)

    train_x = pd.DataFrame()
    train_y = pd.DataFrame()

    fall_data_c = fall_data.copy()
    adl_data_c = adl_data.copy()
    fall_data_c.drop(idxs_test, axis=0, inplace=True)
    adl_data_c.drop(idxs_test_2, axis=0, inplace=True)
    train_x = pd.concat([train_x, fall_data_c, adl_data_c])

    df_fall_c = df_fall.copy()
    df_adl_c = df_adl.copy()
    df_fall_c.drop(idxs_test, axis=0, inplace=True)
    df_adl_c.drop(idxs_test_2, axis=0, inplace=True)
    train_y = pd.concat([train_y, df_fall_c['label'], df_adl_c['label']])
    #print(train_x.shape, train_y.shape)

    classifier_RNN_leave = build_model_RNN()

    #print(np.array(train_y.astype(int)))
    
    history_RNN_leave = classifier_RNN_leave.fit(
        np.array(train_x), np.array(train_y),
        epochs=epochs,
        batch_size = batch_size,
        steps_per_epoch=(train_x.shape[0]) // batch_size,
        validation_data=(np.array(test_x), np.array(test_y)),
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.00003)]
    )

    val_acc_per_fold.append(history_RNN_leave.history['val_accuracy'])
    acc_per_fold.append(history_RNN_leave.history['accuracy'])
    val_loss_per_fold.append(history_RNN_leave.history['val_loss'])
    loss_per_fold.append(history_RNN_leave.history['loss'])

Epoch 1/20
711/711 [==============================] - 30s 40ms/step - loss: 0.4293 - accuracy: 0.8254 - val_loss: 0.4676 - val_accuracy: 0.8203 - lr: 3.0000e-04
Epoch 2/20
711/711 [==============================] - 28s 40ms/step - loss: 0.2818 - accuracy: 0.8953 - val_loss: 0.1386 - val_accuracy: 0.9505 - lr: 3.0000e-04
Epoch 3/20
711/711 [==============================] - 29s 40ms/step - loss: 0.1595 - accuracy: 0.9425 - val_loss: 0.1230 - val_accuracy: 0.9583 - lr: 3.0000e-04
Epoch 4/20
711/711 [==============================] - 29s 40ms/step - loss: 0.1014 - accuracy: 0.9654 - val_loss: 0.1688 - val_accuracy: 0.9505 - lr: 3.0000e-04
Epoch 5/20
711/711 [==============================] - 28s 40ms/step - loss: 0.0676 - accuracy: 0.9762 - val_loss: 0.0176 - val_accuracy: 0.9974 - lr: 3.0000e-04
Epoch 6/20
711/711 [==============================] - 28s 40ms/step - loss: 0.0621 - accuracy: 0.9795 - val_loss: 0.0554 - val_accuracy: 0.9792 - lr: 3.0000e-04
Epoch 7/20
711/711 [==============

In [58]:
val_acc_per_fold, acc_per_fold, val_loss_per_fold, loss_per_fold

([[0.8203125,
   0.9505208134651184,
   0.9583333134651184,
   0.9505208134651184,
   0.9973958134651184,
   0.9791666865348816,
   0.9817708134651184,
   0.9791666865348816,
   0.8046875,
   0.8932291865348816,
   0.9244791865348816,
   0.9427083134651184,
   0.9453125,
   0.9765625,
   0.9791666865348816,
   0.9895833134651184,
   0.984375,
   0.9921875,
   0.9895833134651184,
   0.9895833134651184],
  [0.8164665699005127,
   0.8713550567626953,
   0.9416809678077698,
   0.7873070240020752,
   0.9811320900917053,
   0.9674099683761597,
   0.9502573013305664,
   0.9879931211471558,
   0.9777015447616577,
   0.9691252112388611,
   0.5591766834259033,
   0.7958833575248718,
   0.958833634853363,
   0.9828473329544067,
   0.9691252112388611,
   0.9656946659088135,
   0.9777015447616577,
   0.9965694546699524,
   0.9691252112388611,
   0.994854211807251],
  [0.7664473652839661,
   0.8585526347160339,
   0.9769737124443054,
   0.9835526347160339,
   0.9901315569877625,
   0.976973712444305

In [59]:
import pickle

results = {"val_acc_per_fold": val_acc_per_fold, "acc_per_fold": acc_per_fold, "val_loss_per_fold": val_loss_per_fold, "loss_per_fold": loss_per_fold}

with open('results_model_rnn_leave_one_out.pickle', 'wb') as file:
 pickle.dump(results, file)

In [61]:
s = 0
for fold in val_acc_per_fold:
    print(fold[-1])
    s += fold[-1]
s/30

0.9895833134651184
0.994854211807251
0.9769737124443054
0.9421221613883972
0.9966443181037903
1.0
0.9909090995788574
0.9722921848297119
0.990338146686554
1.0
0.9948979616165161
1.0
0.9204244017601013
0.9973118305206299
0.9974683523178101
0.9935897588729858
1.0
1.0
0.9887217879295349
0.9956616163253784
0.9964285492897034
0.9855371713638306
0.9938650131225586
0.9803600907325745
0.9876543283462524
0.9791666865348816
1.0
1.0
0.9961165189743042
0.9975489974021912


0.9886156737804412

In [26]:
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
classifier_RNN_leave.save("classifier_RNN_leave_one_out")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("classifier_RNN_leave_one_out")

NameError: name 'classifier_RNN' is not defined

## Check - SVM

https://scikit-learn.org/stable/modules/svm.html

**5-fold**

In [23]:
from sklearn import svm, metrics
from tqdm import tqdm

SVC

In [53]:
val_acc_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in tqdm(range(len(sets))):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    #print(np.array(train_y[:][0]).shape)

    clf = svm.SVC(C=0.5, kernel='rbf', gamma='scale')

    history_svm = clf.fit(np.array(train_x), np.array(train_y[:][0]))

    pred=clf.predict(np.array(test_x))
    
    val_acc_per_fold.append(metrics.accuracy_score(np.array(test_y[:][0]), pred))

100%|██████████| 5/5 [00:26<00:00,  5.39s/it]


In [54]:
val_acc_per_fold

[0.9797936371453139,
 0.9810834049871023,
 0.9810834049871023,
 0.9785038693035254,
 0.9793637145313844]

In [55]:
sum(val_acc_per_fold)/5

0.9799656061908857

In [57]:
import pickle

filename = 'svm_98.sav'
pickle.dump(clf, open(filename, 'wb'))
 
#load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

SVC 2

In [61]:
val_acc_per_fold = []
sets = [(fold_1, fold_1_lab), (fold_2, fold_2_lab), (fold_3, fold_3_lab), (fold_4, fold_4_lab), (fold_5, fold_5_lab)]

for n in tqdm(range(len(sets))):
    test_x = sets[n][0]
    test_y = sets[n][1]
    train_x = pd.DataFrame()
    train_y = pd.DataFrame()
    for s in sets[:n] + sets[n+1:]:
        train_x = pd.concat([train_x, s[0]])
        train_y = pd.concat([train_y, s[1]])

    #print(np.array(train_y[:][0]).shape)

    clf = svm.SVC(C=0.7, kernel='rbf', gamma='scale')

    history_svm = clf.fit(np.array(train_x), np.array(train_y[:][0]))

    pred=clf.predict(np.array(test_x))
    
    val_acc_per_fold.append(metrics.accuracy_score(np.array(test_y[:][0]), pred))

100%|██████████| 5/5 [00:23<00:00,  4.72s/it]


In [62]:
val_acc_per_fold

[0.9832330180567498,
 0.9828030954428203,
 0.9845227858985383,
 0.9836629406706793,
 0.9828030954428203]

In [63]:
sum(val_acc_per_fold)/5

0.9834049871023216

In [64]:
import pickle

filename = 'svm_983.sav'
pickle.dump(clf, open(filename, 'wb'))
 
#load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

**Leave one out**

In [72]:
val_acc_per_fold = []

for n in tqdm(range(1, 31)):
    test_x = pd.DataFrame()
    test_y = pd.DataFrame()

    idxs_test = df_fall[df_fall['subject'] == n].index.to_list()
    idxs_test_2 = df_adl[df_adl['subject'] == n].index.to_list()
    test_x = pd.concat([test_x, fall_data.loc[idxs_test], adl_data.loc[idxs_test_2]])
    test_y = pd.concat([test_y, df_fall.loc[idxs_test]['label'], df_adl.loc[idxs_test_2]['label']])
    #print(test_x.shape, test_y.shape)

    train_x = pd.DataFrame()
    train_y = pd.DataFrame()

    fall_data_c = fall_data.copy()
    adl_data_c = adl_data.copy()
    fall_data_c.drop(idxs_test, axis=0, inplace=True)
    adl_data_c.drop(idxs_test_2, axis=0, inplace=True)
    train_x = pd.concat([train_x, fall_data_c, adl_data_c])

    df_fall_c = df_fall.copy()
    df_adl_c = df_adl.copy()
    df_fall_c.drop(idxs_test, axis=0, inplace=True)
    df_adl_c.drop(idxs_test_2, axis=0, inplace=True)
    train_y = pd.concat([train_y, df_fall_c['label'], df_adl_c['label']])
    #print(train_x.shape, train_y.shape)

    clf = svm.SVC(C=0.7, kernel='rbf', gamma='scale')

    history_svm = clf.fit(np.array(train_x), np.array(train_y[:][0]))

    pred=clf.predict(np.array(test_x))
    
    val_acc_per_fold.append(metrics.accuracy_score(np.array(test_y[:][0]), pred))

100%|██████████| 30/30 [02:21<00:00,  4.71s/it]


In [73]:
val_acc_per_fold

[0.9479166666666666,
 0.934819897084048,
 0.993421052631579,
 0.9646302250803859,
 0.9966442953020134,
 0.9563106796116505,
 0.9545454545454546,
 0.9496221662468514,
 0.927536231884058,
 0.9776357827476039,
 0.9566326530612245,
 0.992,
 0.9681697612732095,
 0.9919354838709677,
 0.9898734177215189,
 0.9957264957264957,
 1.0,
 0.9620689655172414,
 0.9924812030075187,
 0.9978308026030369,
 0.9714285714285714,
 0.9008264462809917,
 0.99079754601227,
 0.9525368248772504,
 0.980246913580247,
 0.9940476190476191,
 0.9887640449438202,
 0.9841772151898734,
 0.9669902912621359,
 0.9730392156862745]

In [74]:
sum(val_acc_per_fold)/30

0.9717551974296857

In [75]:
import pickle

filename = 'svm_leave_one_out.sav'
pickle.dump(clf, open(filename, 'wb'))
 
#load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))